In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams
import scipy.stats as st
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
import xgboost as xgb
from sklearn.feature_selection import SelectFromModel # Dimensionality reduction
from sklearn.model_selection import learning_curve, validation_curve, GridSearchCV # Model evaluation
from sklearn.metrics import median_absolute_error, mean_squared_error as MSE

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout, BatchNormalization
from keras.layers.advanced_activations import PReLU
from keras.layers.noise import GaussianDropout
from keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Imputer

In [ ]:
n_folds = 10

def rmse(model):
    kf = KFold(n_folds, shuffle=True, random_state=40).get_n_splits(train.values)
    rmse= np.sqrt(-cross_val_score(model, train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [ ]:
df_usa = pd.read_csv("D:/Columbia U/IEOR 4501 python/Project/kc_house_data.csv")

print("Price Min")
print(df_usa['price'].min())
print("Price Mean")
print(df_usa['price'].mean())
print("Price Median")
print(df_usa['price'].median())
print("Price Max")
print(df_usa['price'].max())
print("Price Std")
print(df_usa['price'].std())

df_usa = df_usa.merge(pd.get_dummies(df_usa.floors, drop_first=True, prefix='Floors'), left_index=True, right_index=True)
df_usa = df_usa.merge(pd.get_dummies(df_usa.waterfront, drop_first=True, prefix='watFront'), left_index=True, right_index=True)
df_usa = df_usa.merge(pd.get_dummies(df_usa.view, drop_first=True, prefix='View'), left_index=True, right_index=True)
df_usa = df_usa.merge(pd.get_dummies(df_usa.condition, drop_first=True, prefix='Cond'), left_index=True, right_index=True)
df_usa = df_usa.merge(pd.get_dummies(df_usa.grade, prefix='Grade'), left_index=True, right_index=True)
df_usa = df_usa.merge(pd.get_dummies(df_usa.bedrooms, drop_first=True, prefix='Bedrooms'), left_index=True, right_index=True)

del df_usa['floors'],
del df_usa['waterfront']
del df_usa['view']
del df_usa['condition']
del df_usa['grade']
del df_usa['bedrooms']


df_usa.drop(['id', 'date'], axis=1, inplace=True)
X = df_usa.drop("price",axis=1).values
y = df_usa["price"].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=3)

In [ ]:
### XGBoost ###

# for c in properties.columns:
#     if properties[c].dtype == 'object':
#         lbl = LabelEncoder()
#         lbl.fit(list(properties[c].values))
#         properties[c] = lbl.transform(list(properties[c].values))
thresh = 5 * 10**(-3)
model = xgb.XGBRegressor()
model.fit(X_train, y_train)
#select features using threshold
selection = SelectFromModel(model, threshold=thresh, prefit=True)
select_X_train = selection.transform(X_train)
# test 
select_X_test = selection.transform(X_test)

xgb = xgb.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7)

xgb.fit(X_train, y_train)

y_hat = xgb.predict(X_test)

xgb.score(X_test,y_test)

n_folds = 10
kfold = KFold(n_splits=n_folds, random_state=42)
cv_results = cross_val_score(model, X_train, y_train, cv= kfold,scoring='r2', n_jobs=-1)





xgb_params = {
    'eta': 0.037,
    'max_depth': 5,
    'subsample': 0.80,
    'objective': 'reg:linear',
    'eval_metric': 'mae',
    'lambda': 0.8,   
    'alpha': 0.4, 
    'base_score': 'y_mean',
    'silent': 1
}

dtrain = xgb.DMatrix(x_train, y_train)
dtest = xgb.DMatrix(x_test)

num_rounds = 250
model = xgb.train(dict(xgb_params, silent=1), dtrain, num_boost_round=num_boost_rounds)

xgb_pred1 = model.predict(dtest)

median_absolute_error(xgb_pred1, y_test)
MSE(xgb_pred1, y_test)

print(pd.DataFrame(xgb_pred1).head())


### Run XGBoost Again ###
xgb_params = {
    'eta': 0.037,
    'max_depth': 10,
    'subsample': 0.80,
    'objective': 'reg:linear',
    'eval_metric': 'mae',
    'silent': 1
}

dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test)

num_boost_rounds = 250

model = xgb.train(dict(xgb_params, silent=1), dtrain, num_boost_round=num_boost_rounds)

xgb_pred2 = model.predict(dtest)

median_absolute_error(xgb_pred2, y_test)

##### COMBINE XGBOOST RESULTS
xgb_pred = XGB1_WEIGHT*xgb_pred1 + (1-XGB1_WEIGHT)*xgb_pred2


del train_df
del x_train
del x_test
del properties
del dtest
del dtrain
del xgb_pred1
del xgb_pred2
gc.collect()

In [ ]:
###  Neural Network  ###
# for c in prop.columns:
#     if prop[c].dtype == 'object':
#         lbl = LabelEncoder()
#         lbl.fit(list(prop[c].values))
#         prop[c] = lbl.transform(list(prop[c].values))
X = df_usa.drop("price",axis=1)
y = df_usa["price"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=3)


# for c in X_train.columns:
#    if X_train[c].dtype == 'object':
#        lbl = LabelEncoder()
#        lbl.fit(list(X_train[c].values))
#        X_train[c] = lbl.transform(list(X_train[c].values))

## Preprocessing
imputer= Imputer()
imputer.fit(X_train.iloc[:, :])
X_train = imputer.transform(X_train.iloc[:, :])
imputer.fit(X_test.iloc[:, :])
X_test = imputer.transform(X_test.iloc[:, :])

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

len_x=int(X_train.shape[1])

# Neural Network
nn = Sequential()
nn.add(Dense(units = 400 , kernel_initializer = 'normal', input_dim = len_x))
nn.add(PReLU())
nn.add(Dropout(.4))
nn.add(Dense(units = 160 , kernel_initializer = 'normal'))
nn.add(PReLU())
nn.add(BatchNormalization())
nn.add(Dropout(.6))
nn.add(Dense(units = 64 , kernel_initializer = 'normal'))
nn.add(PReLU())
nn.add(BatchNormalization())
nn.add(Dropout(.5))
nn.add(Dense(units = 26, kernel_initializer = 'normal'))
nn.add(PReLU())
nn.add(BatchNormalization())
nn.add(Dropout(.6))
nn.add(Dense(1, kernel_initializer='normal'))
nn.compile(loss='mse', optimizer=Adam(lr=4e-3, decay=1e-4))

nn.fit(np.array(X_train), np.array(y_train), batch_size = 32, epochs = 100, verbose=2)
y_pred_ann = nn.predict(X_test)

nn_pred = y_pred_ann.flatten()

median_absolute_error(nn_pred, y_test)


# Cleanup
del train
del prop
del sample
del X_train
del X_test
del df_train
del df_test
del y_pred_ann
gc.collect()
